In [1]:
import tensorflow as tf
import cv2 as cv
import numpy as np

In [ ]:
model_path = r'models\face_detection_yunet_2023mar.onnx'
yolo_model = tf.lite.Interpreter(r'models\ob_track_fp32.tflite')
yolo_model.allocate_tensors()
face_detector = cv.FaceDetectorYN.create(model_path, "", (320, 320))

In [12]:
import cv2
import numpy as np
import tensorflow.lite as tflite

def yolo_bbox(img_path, img_size=(640, 640)):

    interpreter = tflite.Interpreter(r'models\yolo11n_float32.tflite')
    interpreter.allocate_tensors()

    inp_details = interpreter.get_input_details()[0]
    out_details = interpreter.get_output_details()[0]

    original_img = cv2.imread(img_path)
    resized_img = cv2.resize(original_img, img_size)
    img = resized_img.astype(np.float32) / 255.0
    img_p = np.expand_dims(img, axis=0)

    interpreter.set_tensor(inp_details['index'], img_p)
    interpreter.invoke()
    out = interpreter.get_tensor(out_details['index']).squeeze().T

    boxes = out[:, 0:4]
    class_probs = out[:, 4:]
    class_ids = np.argmax(class_probs, axis=1)
    confidence_scores = np.max(class_probs, axis=1)

    confidence_threshold = 0.5
    mask = (class_ids == 0) & (confidence_scores > confidence_threshold)

    filtered_boxes = boxes[mask]
    filtered_scores = confidence_scores[mask]

    if filtered_boxes.size == 0:
        print("No detections found.")
        return

    image_height, image_width = img_size
    x_center, y_center, w, h = filtered_boxes[:, 0], filtered_boxes[:, 1], filtered_boxes[:, 2], filtered_boxes[:, 3]
    x1 = (x_center - w / 2) * image_width
    y1 = (y_center - h / 2) * image_height
    x2 = (x_center + w / 2) * image_width
    y2 = (y_center + h / 2) * image_height

    nms_threshold = 0.5
    boxes_nms = np.stack([x1, y1, x2 - x1, y2 - y1], axis=1).astype(np.float32).tolist()
    indices = cv2.dnn.NMSBoxes(boxes_nms, filtered_scores.tolist(), confidence_threshold, nms_threshold)

    for i in indices.flatten():
        x1_i, y1_i, x2_i, y2_i = int(x1[i]), int(y1[i]), int(x2[i]), int(y2[i])
        cv2.rectangle(resized_img, (x1_i, y1_i), (x2_i, y2_i), (0, 255, 0), 2)
        cv2.putText(resized_img, f"Person: {filtered_scores[i]:.2f}", (x1_i, y1_i - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2.imshow('Detection Results', resized_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [16]:
img_path = r"C:\Users\s9554\Downloads\kids-in-classroom.jpg"

In [17]:
yolo_bbox(img_path)